In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv("./data/pre_processed_credit.csv")
df.head(2)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,loan_to_income_ratio,loan_to_emp_length_ratio,...,medium,large,very large,A,B,C,D,E,F,G
0,21,9600,5.0,1000,11.14,0,0.10,2,0.104167,0.005000,...,0,0,0,0,1,0,0,0,0,0
1,25,9600,1.0,5500,12.87,1,0.57,3,0.572917,0.000182,...,1,0,0,0,0,1,0,0,0,0


In [3]:
df.columns

Index(['person_age', 'person_income', 'person_emp_length', 'loan_amnt',
       'loan_int_rate', 'loan_status', 'loan_percent_income',
       'cb_person_cred_hist_length', 'loan_to_income_ratio',
       'loan_to_emp_length_ratio', 'int_rate_to_loan_amt_ratio', 'MORTGAGE',
       'OTHER', 'OWN', 'RENT', 'DEBTCONSOLIDATION', 'EDUCATION',
       'HOMEIMPROVEMENT', 'MEDICAL', 'PERSONAL', 'VENTURE', 'N', 'Y', '20-25',
       '26-35', '36-45', '46-55', '56-65', 'low', 'low-middle', 'middle',
       'high-middle', 'high', 'small', 'medium', 'large', 'very large', 'A',
       'B', 'C', 'D', 'E', 'F', 'G'],
      dtype='object')

In [4]:
y = df["loan_status"]

In [5]:
df.drop(['loan_status'],  axis =1 ,inplace=True)

In [6]:
df.columns

Index(['person_age', 'person_income', 'person_emp_length', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
       'loan_to_income_ratio', 'loan_to_emp_length_ratio',
       'int_rate_to_loan_amt_ratio', 'MORTGAGE', 'OTHER', 'OWN', 'RENT',
       'DEBTCONSOLIDATION', 'EDUCATION', 'HOMEIMPROVEMENT', 'MEDICAL',
       'PERSONAL', 'VENTURE', 'N', 'Y', '20-25', '26-35', '36-45', '46-55',
       '56-65', 'low', 'low-middle', 'middle', 'high-middle', 'high', 'small',
       'medium', 'large', 'very large', 'A', 'B', 'C', 'D', 'E', 'F', 'G'],
      dtype='object')

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=42)

In [8]:
X_train.shape

(19182, 43)

In [9]:
X_test.shape

(9448, 43)

In [10]:
y_train.shape

(19182,)

In [11]:
y_test.shape

(9448,)

# Normalize data

In [12]:
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform testing dataabs
X_test_norm = norm.transform(X_test)

In [13]:
X_train = X_train_norm
X_test = X_test_norm

# LOGISTIC REGRESSION

In [14]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(penalty= 'l1', solver='saga')

In [15]:
param_grid = [    
    {#'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20)
    }
]

In [16]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(LR, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

In [17]:
best_clf = clf.fit(X_train,y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [18]:
best_clf.best_estimator_

LogisticRegression(C=78.47599703514607, penalty='l1', solver='saga')

In [19]:
print (f'Accuracy: {best_clf.score(X_train,y_train):.3f}')

Accuracy: 0.879


In [20]:
y_pred_LR = best_clf.predict(X_test)

In [21]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_LR)

array([[7016,  349],
       [ 861, 1222]], dtype=int64)

In [22]:
from sklearn.metrics import classification_report
print("Classification report - \n", classification_report(y_test,y_pred_LR))

Classification report - 
               precision    recall  f1-score   support

           0       0.89      0.95      0.92      7365
           1       0.78      0.59      0.67      2083

    accuracy                           0.87      9448
   macro avg       0.83      0.77      0.79      9448
weighted avg       0.87      0.87      0.87      9448



### penalty= 'l2', solver='saga'

In [23]:
LR = LogisticRegression(penalty= 'l2', solver='liblinear')
LR.fit(X_train,y_train)

LogisticRegression(solver='liblinear')

In [24]:
y_pred_LR = LR.predict(X_test)

In [25]:
confusion_matrix(y_test,y_pred_LR)

array([[7016,  349],
       [ 890, 1193]], dtype=int64)

In [26]:
print("Classification report - \n", classification_report(y_test,y_pred_LR))

Classification report - 
               precision    recall  f1-score   support

           0       0.89      0.95      0.92      7365
           1       0.77      0.57      0.66      2083

    accuracy                           0.87      9448
   macro avg       0.83      0.76      0.79      9448
weighted avg       0.86      0.87      0.86      9448



In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_LR)

0.868861134631668

In [28]:
from joblib import Parallel, delayed 
import joblib 
  
  
# Save the model as a pickle in a file 
joblib.dump(LR, './model/Logistic_model.pkl') 

['./model/Logistic_model.pkl']

# Decision Tree

In [29]:
from sklearn.tree import DecisionTreeClassifier#for checking testing results
from sklearn.metrics import classification_report, confusion_matrix#for visualizing tree 

In [30]:
# Defining the decision tree algorithm
dtree=DecisionTreeClassifier(max_depth=100)
dtree.fit(X_train,y_train)
print('Decision Tree Classifier Created')

Decision Tree Classifier Created


In [31]:
# Predicting the values of test data
y_pred_DT = dtree.predict(X_test)
print("Classification report - \n", classification_report(y_test,y_pred_DT))

Classification report - 
               precision    recall  f1-score   support

           0       0.93      0.92      0.93      7365
           1       0.73      0.76      0.75      2083

    accuracy                           0.89      9448
   macro avg       0.83      0.84      0.84      9448
weighted avg       0.89      0.89      0.89      9448



In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_DT)

array([[6781,  584],
       [ 490, 1593]], dtype=int64)

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_DT)

0.8863251481795089

In [34]:
from joblib import Parallel, delayed 
import joblib 
  
  
# Save the model as a pickle in a file 
joblib.dump(dtree, './model/decisiontree_model.pkl') 

['./model/decisiontree_model.pkl']

# RANDOM FOREST

In [35]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=50)

#RF.fit(X_train_scaled,y_train)
RF.fit(X_train,y_train)

RandomForestClassifier(n_estimators=50)

In [36]:
#y_pred_RF = RF.predict(X_test_scaled)
y_pred_RF = RF.predict(X_test)

In [37]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_RF)

array([[7288,   77],
       [ 586, 1497]], dtype=int64)

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_RF)

0.9298264182895851

In [39]:
print("Classification report - \n", classification_report(y_test,y_pred_RF))

Classification report - 
               precision    recall  f1-score   support

           0       0.93      0.99      0.96      7365
           1       0.95      0.72      0.82      2083

    accuracy                           0.93      9448
   macro avg       0.94      0.85      0.89      9448
weighted avg       0.93      0.93      0.93      9448



In [40]:
from joblib import Parallel, delayed 
import joblib 
  
  
# Save the model as a pickle in a file 
joblib.dump(RF, './model/randomforest_model.pkl') 

['./model/randomforest_model.pkl']

# Neural Network

In [41]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import keras

In [42]:
# define the keras model
import tensorflow as tf

optimizer = tf.keras.optimizers.Adam(lr=0.001)
model = Sequential()
model.add(Dense(32, input_dim=43, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=200, batch_size=1000)
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/200


C:\Users\My Pc\AppData\Roaming\Python\Python38\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


20/20 [==============================] - 1s 3ms/step - loss: 0.6900 - accuracy: 0.5529
Epoch 2/200
20/20 [==============================] - 0s 3ms/step - loss: 0.5893 - accuracy: 0.7254
Epoch 3/200
20/20 [==============================] - 0s 2ms/step - loss: 0.5481 - accuracy: 0.7640
Epoch 4/200
20/20 [==============================] - 0s 3ms/step - loss: 0.5242 - accuracy: 0.7772
Epoch 5/200
20/20 [==============================] - 0s 2ms/step - loss: 0.5005 - accuracy: 0.7880
Epoch 6/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4761 - accuracy: 0.8009
Epoch 7/200
20/20 [==============================] - 0s 2ms/step - loss: 0.4551 - accuracy: 0.8150
Epoch 8/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4356 - accuracy: 0.8197
Epoch 9/200
20/20 [==============================] - 0s 2ms/step - loss: 0.4160 - accuracy: 0.8325
Epoch 10/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4065 - accuracy: 0.8358
Epoch 11/200
20/20 [=

20/20 [==============================] - 0s 2ms/step - loss: 0.2655 - accuracy: 0.9104
Epoch 84/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2650 - accuracy: 0.9126
Epoch 85/200
20/20 [==============================] - 0s 4ms/step - loss: 0.2614 - accuracy: 0.9115
Epoch 86/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2633 - accuracy: 0.9115
Epoch 87/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2616 - accuracy: 0.9114
Epoch 88/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2619 - accuracy: 0.9136
Epoch 89/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2610 - accuracy: 0.9117
Epoch 90/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2634 - accuracy: 0.9134
Epoch 91/200
20/20 [==============================] - 0s 2ms/step - loss: 0.2610 - accuracy: 0.9113
Epoch 92/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2572 - accuracy: 0.9130
Epoch 93/200


20/20 [==============================] - 0s 3ms/step - loss: 0.2380 - accuracy: 0.9192
Epoch 165/200
20/20 [==============================] - 0s 4ms/step - loss: 0.2366 - accuracy: 0.9204
Epoch 166/200
20/20 [==============================] - 0s 4ms/step - loss: 0.2342 - accuracy: 0.9210
Epoch 167/200
20/20 [==============================] - 0s 4ms/step - loss: 0.2334 - accuracy: 0.9213
Epoch 168/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2336 - accuracy: 0.9209
Epoch 169/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2338 - accuracy: 0.9215
Epoch 170/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2341 - accuracy: 0.9211
Epoch 171/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2337 - accuracy: 0.9212
Epoch 172/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2359 - accuracy: 0.9203
Epoch 173/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2334 - accuracy: 0.9220
Epoc

In [43]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)

In [44]:
loss

0.2336268126964569

In [45]:
acc

0.9192421436309814

In [46]:
predict_x=model.predict(X_test) 
yhat_classes=np.argmax(predict_x,axis=1)

In [47]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print("Accuracy: %f" % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes, average='weighted', labels=np.unique(yhat_classes))
print("Precision: %f" % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes, average='weighted', labels=np.unique(yhat_classes))
print("Recall: %f" % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes, average='weighted', labels=np.unique(yhat_classes))
print("F1 score: %f" % f1)

# confusion matrix
matrix = confusion_matrix(y_test, yhat_classes)
print(matrix)

Accuracy: 0.779530
Precision: 0.779530
Recall: 1.000000
F1 score: 0.876108
[[7365    0]
 [2083    0]]


In [48]:
from joblib import Parallel, delayed 
import joblib 
  
  
# Save the model as a pickle in a file 
joblib.dump(model, './model/nn_model.pkl') 

INFO:tensorflow:Assets written to: ram://773a7816-7b95-423b-b16e-3d89159325c2/assets


['./model/nn_model.pkl']